In [ ]:
from pydrake.all import (
    RgbdSensor,
    DiagramBuilder,
    AddMultibodyPlantSceneGraph,
    Parser,
    ConnectDrakeVisualizer,
    FindResourceOrThrow,
    Simulator,
    RigidTransform,
    RollPitchYaw,
    DepthCameraProperties,
    MakeRenderEngineVtk, RenderEngineVtkParams,
    RenderLabel,
    world_model_instance,
)

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

In [ ]:
def xyz_rpy_deg(xyz, rpy_deg):
    rpy_deg = np.asarray(rpy_deg)
    return RigidTransform(RollPitchYaw(rpy_deg * np.pi / 180), xyz)

In [ ]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder)

sdf_file = FindResourceOrThrow("drake/manipulation/models/iiwa_description/sdf/iiwa14_no_collision.sdf")
iiwa_1 = Parser(plant).AddModelFromFile(sdf_file, model_name="iiwa_1")
plant.WeldFrames(
    plant.world_frame(), plant.GetFrameByName("iiwa_link_0", iiwa_1),
    X_AB=xyz_rpy_deg([0, 0, 0], [0, 0, 0]))

iiwa_2 = Parser(plant).AddModelFromFile(sdf_file, model_name="iiwa_2")
plant.WeldFrames(
    plant.world_frame(), plant.GetFrameByName("iiwa_link_0", iiwa_2),
    X_AB=xyz_rpy_deg([0, 1, 0], [0, 0, 0]))

ConnectDrakeVisualizer(builder, scene_graph)

scene_graph.AddRenderer("renderer", MakeRenderEngineVtk(RenderEngineVtkParams()))
depth_prop = DepthCameraProperties(
    width=640, height=480, fov_y=np.pi/4, renderer_name="renderer",
    z_near=0.01, z_far=10.)

world_id = plant.GetBodyFrameIdOrThrow(plant.world_body().index())
X_WB = xyz_rpy_deg([4, 0, 0], [-90, 0, 90])
sensor = RgbdSensor(world_id, X_PB=X_WB, properties=depth_prop)
builder.AddSystem(sensor)
builder.Connect(scene_graph.get_query_output_port(), sensor.query_object_input_port())

plant.Finalize()
diagram = builder.Build()
diagram_context = diagram.CreateDefaultContext()

In [ ]:
sensor_context = sensor.GetMyMutableContextFromRoot(diagram_context)

In [ ]:
Simulator(diagram).Initialize()

In [ ]:
reserved_labels = [
    RenderLabel.kDoNotRender, RenderLabel.kDontCare, RenderLabel.kEmpty, RenderLabel.kUnspecified]

In [ ]:
def colorize_labels(image, colors=None, bg_color=[0, 0, 0]):
    if colors is None:
        cc = mpl.colors.ColorConverter()
        color_cycle = plt.rcParams["axes.prop_cycle"]
        colors = np.array([cc.to_rgb(c["color"]) for c in color_cycle])
    image = np.squeeze(image)
    background = np.zeros(image.shape[:2], dtype=bool)
    for label in reserved_labels:
        background |= image == int(label)
    color_image = colors[image % len(colors)]
    color_image[background] = bg_color
    return color_image

In [ ]:
color = sensor.color_image_output_port().Eval(sensor_context).data
label = sensor.label_image_output_port().Eval(sensor_context).data
_, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(color)
ax[1].imshow(colorize_labels(label))

In [ ]:
# Change to model instances. 0 

In [ ]:
def